In [1]:
from sentence_transformers import SentenceTransformer
from PIL import Image

# Load CLIP model
model = SentenceTransformer("clip-ViT-B-32")

/Users/yujian/Documents/workspace/advanced-rag-applications-with-vector-databases-3886256/ch2/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/yujian/Documents/workspace/advanced-rag-applications-with-vector-databases-3886256/ch2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import os
from glob import glob

In [3]:
images_path="../images"

image_paths = glob(os.path.join(images_path, '**/*.jpeg'), recursive=True)

def generate_clip_embeddings(image_path, model, default_size= (128, 128)):
    image = Image.open(image_path).resize(default_size)
    embedding = model.encode(image)

    return embedding

In [4]:
import faiss
import numpy as np

In [5]:
def load_faiss_index(index_path):
    index = faiss.read_index(index_path)
    with open(index_path + '.paths', 'r') as f:
        image_paths = [line.strip() for line in f]
    print(f"Index loaded from {index_path}")
    return index, image_paths

In [6]:
index_path = "./indices/vector.index"
index, image_paths = load_faiss_index(index_path)

Index loaded from ./indices/vector.index


In [9]:
def retrieve_similar_images(query_path, model, index, image_paths, top_k=3):
    
    query_embedding = generate_clip_embeddings(query_path, model)

    query_embedding = query_embedding.astype(np.float32).reshape(1, -1)

    distances, indices = index.search(query_embedding, top_k)

    retrieved_images = [image_paths[int(idx)] for idx in indices[0]]

    return query_path, retrieved_images, distances


In [10]:
qp, retrieved_image_paths, distances = retrieve_similar_images("../images/AdobeStock_92071640.jpeg", model, index, image_paths)

: 